- ID / Dateiname / Identifier
- Datum des Postprocessings (genauer kriegen wirs halt nicht)
- Art der Annotation (händisch, automatisch)
	- Falls Händisch: Wer? Projekt?
	- Falls Automatisch: Architektur? Modell?
- Version (?)
- Dokument-Infos
	- Erstellungsdatum

Identifier:
<fileDesc>
 <titleStmt>
  <title>HGB-Identifier</title>
 </titleStmt>
 <publicationStmt>
  <p>Processed and released to be used privately</p>
 </publicationStmt>
 <sourceDesc>
  <p>Identifier converted to Staatsarchiv ID</p>
 </sourceDesc>
</fileDesc>

Datum des Postprocessings:
<revisionDesc status="embargoed">
 <change when="1991-11-11" who="#LB"> deleted chapter 10 </change>
</revisionDesc>

Händisches Annotieren:
<titleStmt>
 <title>Capgrave's Life of St. John Norbert: a machine-readable transcription</title>
 <respStmt>
  <resp>compiled by</resp>
  <name>P.J. Lucas</name>
 </respStmt>
</titleStmt>

Nicht-händisches Annotatieren:
<appInfo>
 <application version="1.24" ident="Xaira">
  <label>XAIRA Indexer</label>
  <ptr target="#P1"/>
 </application>
</appInfo>
(innerhalb der encodingDesc)

Erstelldatum des originalen (naja, nicht ganz korrekt, das ist kompliziert, aber wir machen ja keine Edition) Dokuments:
<creation>
 <date when="1988-07-10">10 July 1988</date>
</creation>

oder, vielleicht passender:
<settingDesc>
  <setting>
   <name>Paris, France</name>
   <time>Late 19th century</time>
  </setting>
 </settingDesc>


In [ ]:
import glob
import os
import pprint as pp

filepaths = []
for infile in glob.glob("../auto_tagged/*.xml"):
    # fix sorting
    basename = os.path.basename(infile)
    num = int(basename.replace(".xml", ""))
    filepaths.append((num, infile))
filepaths = sorted(filepaths, key=lambda x: x[0])

In [6]:
import glob
import os
import pprint as pp

filepaths = sorted(glob.glob("../outfiles/*.xml"))

In [11]:
import json

with open("../consistent_data.json", mode="r", encoding="utf8") as inf:
    cons = json.load(inf)

testfiles = cons["test"]

print(testfiles)

['admin_004_HGB_1_013_012_066.xml', 'admin_009_HGB_1_024_108_011.xml', 'admin_018_HGB_1_051_086_076.xml', 'admin_072_HGB_1_177_032_049.xml', 'admin_085_HGB_1_203_011_031.xml', 'admin_HGB_Exp_11_009_HGB_1_010_067_052.xml', 'admin_HGB_Exp_11_014_HGB_1_010_119_004.xml', 'admin_HGB_Exp_11_031_HGB_1_058_020_022.xml', 'admin_HGB_Exp_11_109_HGB_1_151_016_030.xml', 'admin_HGB_Exp_11_111_HGB_1_153_016_026.xml', 'admin_HGB_Exp_11_112_HGB_1_154_040_010.xml', 'admin_HGB_Exp_5_044_HGB_1_122_018_011.xml', 'admin_HGB_Exp_5_047_HGB_1_133_027_010.xml', 'admin_HGB_Exp_5_052_HGB_1_145_027_014.xml', 'admin_HGB_Exp_5_054_HGB_1_146_025_019.xml', 'admin_HGB_Exp_5_078_HGB_1_189_054_014.xml', 'admin_HGB_Exp_6_017_HGB_1_050_037_030.xml', 'admin_HGB_Exp_6_035_HGB_1_063_109_029.xml', 'admin_HGB_Exp_6_047_HGB_1_085_027_032.xml', 'admin_HGB_Exp_6_055_HGB_1_113_024_004.xml', 'admin_HGB_Exp_6_060_HGB_1_145_012_022.xml', 'admin_HGB_Exp_6_065_HGB_1_154_027_008.xml', 'admin_HGB_Exp_6_082_HGB_1_189_063_004.xml', 'admin_H

In [3]:
import csv

year_dict = {}
with open("../hgb_resources/hgb_page_year.csv", mode="r", encoding="utf8") as inf:
    year_reader = csv.DictReader(inf)
    for row in year_reader:
        year_dict[row["hgb_page"]] = row["year"]

In [8]:
from lxml import etree as et
from datetime import date

for filepath in filepaths:
    root = et.parse(filepath).getroot()
    metadata = et.Element("Metadata")
    root.insert(0, metadata)

    fileDesc = et.SubElement(metadata, "FileDesc")
    title = et.SubElement(fileDesc, "Title")
    title.text = os.path.basename(os.path.basename(filepath))
    
    settingDesc = et.SubElement(metadata, "SettingDesc")
    time = et.SubElement(settingDesc, "Time")
    basename = "_".join(os.path.basename(filepath).split("_")[-5:])
    try:
        time.text = year_dict[basename.replace(".xml", "")]
    except:
        print(f"WARNING: No year entry found for {basename}!")

    changelog = et.SubElement(metadata, "ChangeLog")
    change = et.SubElement(changelog, "Change", when=str(date.today()), who="IP")
    change.text = "Postprocessed. Annotated by a human, TODO to write here who annotated it."

    new_filepath = filepath
    et.ElementTree(root).write(new_filepath, pretty_print=True, xml_declaration=True, encoding='UTF-8')

In [32]:
# spezifisch für die automatisch getaggten Dateien

from lxml import etree as et
from datetime import date

for (s, filepath), testfile in zip(filepaths, testfiles):
    cleaned_testfile = "_".join(testfile.split("_")[-5:])

    root = et.parse(filepath).getroot()
    metadata = et.Element("Metadata")
    root.insert(0, metadata)

    fileDesc = et.SubElement(metadata, "FileDesc")
    title = et.SubElement(fileDesc, "Title")
    title.text = os.path.basename(cleaned_testfile)
    
    settingDesc = et.SubElement(metadata, "SettingDesc")
    time = et.SubElement(settingDesc, "Time")
    time.text = year_dict[cleaned_testfile.replace(".xml", "")]

    changelog = et.SubElement(metadata, "ChangeLog")
    change = et.SubElement(changelog, "Change", when=str(date.today()), who="IP")
    change.text = "Postprocessing of automatically annotated document. Annotation was performed by using Flair Recursive Algorithm developed by IP, using model exp_A."

    new_filepath = os.path.join(os.path.split(filepath)[0], cleaned_testfile)
    et.ElementTree(root).write(new_filepath, pretty_print=True, xml_declaration=True, encoding='UTF-8')